In [11]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
import re
import string
import spacy
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import heapq
import yaml
from nltk.stem import WordNetLemmatizer, PorterStemmer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax
import numpy as np
from array import array
import copy

nlp = spacy.load('en_core_web_lg')

import time

start = time.time()

questions = pd.read_csv('questions.csv')
questions = questions.drop(columns=['id', 'qid1', 'qid2'])

In [12]:
from tqdm import tqdm

resulting_df = pd.DataFrame(columns=['question1', 'question2', 'result', 'transcribed_result'])


for index, row in tqdm(questions.iterrows(), total=questions.shape[0]):
    question_a = row['question1']
    question_b = row['question2']
    search_doc_no_stop_words = nlp(str(question_a))
    
    result = 0
    try:
      result = 1 if search_doc_no_stop_words.similarity(nlp(str(question_b))) > 0.60 else 0
    except:
      result = 0
    
    resulting_df = resulting_df.append({'question1': question_a, 'question2': question_b, 'result': result, 'transcribed_result': result}, ignore_index=True)

resulting_df.to_csv('questions-spacy-full.csv')

 18%|█▊        | 74311/404351 [13:43<1:12:19, 76.05it/s] /var/folders/x0/f6tjyw_s7vqbx5gcpvt0z31w0000gn/T/ipykernel_67381/2944887594.py:13: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  result = 1 if search_doc_no_stop_words.similarity(nlp(str(question_b))) > 0.60 else 0
100%|██████████| 404351/404351 [2:08:16<00:00, 52.54it/s]  


In [13]:
resulting_df

,question1,question2,result,transcribed_result
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,1,1
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,1,1
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,1,1
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,1,1
...,...,...,...,...
404346,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,1,1
404347,Do you believe there is life after death?,Is it true that there is life after death?,1,1
404348,What is one coin?,What's this coin?,1,1
404349,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,0


In [14]:
test_questions = questions.reset_index(drop=True)
test_questions['is_match'] = test_questions['is_duplicate'] == resulting_df['transcribed_result']
test_questions['is_match'] = test_questions['is_match'].astype(int)

test_questions.to_csv('questions-spacy-with-match.csv')

In [15]:
count_ones = (test_questions['is_match'] == 1).sum()
total_count = len(test_questions['is_match'])
percentage_ones = (count_ones / total_count) * 100

print("Percentage of 1s: {:.2f}%".format(percentage_ones))

Percentage of 1s: 45.82%
